# Import bibliotek i danych

In [90]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn import metrics
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from operator import itemgetter
import time
from xgboost import XGBRegressor
from sklearn import metrics
from sklearn import neighbors
import statsmodels.formula.api as smf
from statsmodels.stats.outliers_influence import variance_inflation_factor 
from sklearn.model_selection import KFold
from sklearn.metrics import *
from sklearn.linear_model import LinearRegression
from scipy import stats
from sklearn import linear_model
import pickle
with open("model_kNN.p", "rb") as fp:
    modelkNN = pickle.load(fp)
with open("model_OLS.p", "rb") as fp:
    modelOLS = pickle.load(fp)
with open("model_XGBoost.p", "rb") as fp:
    modelXGBoost = pickle.load(fp)
    
df = pd.read_csv('data_eda.csv')
df=df.rename(columns={'Unnamed: 0': 'index_0'})

Tworzymy dataframe z predykcjami wszystkich modeli

In [92]:
predNames = []
true = df[target].sort_index()
for result in [modelXGBoost, modelkNN, modelOLS]:
    # Tworzymy wektor predykcji tworząc serię pandasa, a następnie sortujemy indeks
    pred = pd.Series(sum(result["predictions"], []), index=sum(result["indices"], [])).sort_index()
    # Dodajmy kolumnę z predykcjami na podstawie nazwy z pliku
    # Wyczyścimy nazwy modeli aby zlikwidować spacje i pozbyć się niepotrzebnych składników
    cleanName = (result['name']).replace(" ", "_").split("_")[0]
    predNames.append(f"pred_{cleanName}")
    df[f"pred_{cleanName}"] = pred

In [93]:
df[predNames].corr()

,pred_XGBoost,pred_KNN,pred_OLS
pred_XGBoost,1.000000,0.841211,0.844737
pred_KNN,0.841211,1.000000,0.779810
pred_OLS,0.844737,0.779810,1.000000


Najmniejsza korelacja między KNN i OLS, dla reszty dość wysoka.

In [94]:
df[predNames].sample(10)

,pred_XGBoost,pred_KNN,pred_OLS
5461,3.667854,3.696292,3.701928
119,3.752836,3.893834,3.730671
6429,4.030631,3.988778,4.230372
3902,3.987619,4.182165,3.973627
1012,3.917302,3.928595,3.900619
4662,4.304137,4.062801,4.207721
2019,3.754264,3.885266,3.917883
4668,4.339694,4.283528,4.102498
4294,4.287140,4.140000,4.446698
6296,3.682573,3.942500,3.754338


Ale nadal widać różnice w predykcjach, sprawdźmy jakość predykcji średniej z modeli:

In [95]:
df["pred_mean"] = df[['pred_XGBoost','pred_KNN','pred_OLS']].mean(axis=1)

In [98]:
metrics.mean_squared_error(df['stars'], df['pred_mean'])**0.5

0.1929641178716523

Jakość predykcji lepsza niż dla kNN i OLS osobno, ale gorsza niż dla XGBoost, więc spróbujmy zmienić wagi na współczynniki z regresji:

In [111]:
from sklearn import linear_model

from statsmodels.api import OLS
OLS(df['stars'],df[['pred_XGBoost','pred_KNN','pred_OLS']]).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                  stars   R-squared (uncentered):                   0.998
Model:                            OLS   Adj. R-squared (uncentered):              0.998
Method:                 Least Squares   F-statistic:                          1.015e+06
Date:                Mon, 22 Apr 2024   Prob (F-statistic):                        0.00
Time:                        16:29:08   Log-Likelihood:                          1775.5
No. Observations:                6930   AIC:                                     -3545.
Df Residuals:                    6927   BIC:                                     -3525.
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
pred_XGBoost     0.7733      0.021     36.250      0.000       0.732       0.815
pred_KNN         0.1100      0.019      5.805      0.000       0.073       0.147
pred_OLS         0.1172      0.019      6.118      0.000       0.080       0.155
==============================================================================
Omnibus:                      346.896   Durbin-Watson:                   1.998
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              646.315
Skew:                          -0.380   Prob(JB):                    4.51e-141
Kurtosis:                       4.288   Cond. No.                         78.9
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [112]:
df["pred_regg"] = df['pred_XGBoost']*0.7733+df['pred_KNN']*0.1100+df['pred_OLS']*0.1172
metrics.mean_squared_error(df['stars'], df['pred_regg'])**0.5

0.18728083923943667

Ostateczny wynik lepszy niż dla samego XGBoost: *0.18728*